# Training script

## Dependencies

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.backends.cudnn as cudnn
import torchvision
import torch.autograd as autograd

import imp
import os
import sys
print(sys.getdefaultencoding())
import math
import time
import random
import shutil

from tqdm import tqdm
import numpy as np

import os.path

utf-8


## Data loading

In [2]:
# Data loader
import os
import os.path as op
import cv2
from collections import namedtuple
from random import shuffle
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch


class DataLoader(object):
    def __init__(self, data_dir, size=(312, 312)):
        self.data_dir = data_dir
        self.__find_images()
        self.shuffle = shuffle

        self.size = size
        sample = cv2.imread(self.images[0])
        self.img_size = (sample.shape[0], sample.shape[1])
        self.__compute_offsets()
        
        self.batch_num = len(self.images)
        
    def __compute_offsets(self):
        self.row_nums = int(np.ceil(self.img_size[0] / float(self.size[0])))
        self.col_nums = int(np.ceil(self.img_size[1] / float(self.size[1])))
        print('rows', self.row_nums)
        print('cols', self.col_nums)
        
        offset_w = int((self.col_nums * self.size[1] - self.img_size[1]) / (2 * (self.col_nums - 1)))
        self.margin_w = int(offset_w / 3)
        
        offset_h = int((self.row_nums * self.size[0] - self.img_size[0]) / (2 * (self.row_nums - 1)))
        self.margin_h = int(offset_h / 3)
        print('offset w', offset_w)
        print('offset h', offset_h)
    
        self.offsets_w = [i * (self.size[1] - offset_w) for i in range(self.col_nums)]
        self.offsets_w[-1] = self.img_size[1] - self.size[1]
        
        self.offsets_h = [i * (self.size[0] - offset_h) for i in range(self.row_nums)]
        self.offsets_h[-1] = self.img_size[0] - self.size[0]
        
    def __split_sample(self, sample):
        splitted = []
        
        for offset_h in self.offsets_h:
            for offset_w in self.offsets_w:
                h1, h2 = offset_h, offset_h + self.size[0]
                w1, w2 = offset_w, offset_w + self.size[1]
                splitted.append(sample[h1: h2, w1: w2])
        return splitted
    
    def __compute_merge_offsets(self, offset_h, offset_w, pred):
        if offset_h == 0:
            h1 = 0
            h2 = self.size[0] - self.margin_h
            pred = pred[:, :pred.shape[1]-self.margin_h, :]
        elif offset_h == self.offsets_h[-1]:
            h1 = offset_h + self.margin_h
            h2 = offset_h + self.size[0]
            pred = pred[:, self.margin_h:, :]
        else:
            h1 = offset_h + self.margin_h
            h2 = h1 + self.size[0] - 2*self.margin_h
            pred = pred[:, self.margin_h: pred.shape[1]-self.margin_h, :]
            
        if offset_w == 0:
            w1 = 0
            w2 = self.size[1] - self.margin_w
            pred = pred[:, :, :pred.shape[2]-self.margin_w]
        elif offset_w == self.offsets_w[-1]:
            w1 = offset_w + self.margin_w
            w2 = offset_w + self.size[1]
            pred = pred[:, :, self.margin_w:]
        else:
            w1 = offset_w + self.margin_w
            w2 = w1 + self.size[1] - 2*self.margin_w
            pred = pred[:, :, self.margin_w:pred.shape[2]-self.margin_w]
        return h1, h2, w1, w2, pred
    
    def recreate_mask(self, predicted):
        final = np.ones((3, self.img_size[0], self.img_size[1])) * np.NINF
        
        for offset_h in self.offsets_h:
            for offset_w in self.offsets_w:
                
                h1, h2, w1, w2, pred = self.__compute_merge_offsets(offset_h, offset_w, predicted.pop(0)[0])
        
                final[:, h1: h2, w1: w2] = np.maximum(pred, final[:, h1: h2, w1: w2])
        return final
        
    def __find_images(self):
        self.images = [op.join(self.data_dir, path) for path in os.listdir(self.data_dir)]
    
    def __load_data(self, sample):
        inputs = [] 
        name = op.splitext(op.basename(sample))[0]
        
        raw = cv2.imread(sample, 0)
        splitted = self.__split_sample(raw)
            
        for train_sample in splitted:
            inputs.append(torch.from_numpy(np.expand_dims(np.expand_dims(np.array(train_sample) / 225., 0), 0)).float())
            
        return name, inputs
    
    def __iter__(self):
        if self.shuffle:
            shuffle(self.images)
        for i in range(len(self.images)):
            yield self.__load_data(self.images[i])

## Layers

In [3]:
import torch.nn as nn


def center_crop(layer, max_height, max_width):
    #https://github.com/Lasagne/Lasagne/blob/master/lasagne/layers/merge.py#L162
    #Author does a center crop which crops both inputs (skip and upsample) to size of minimum dimension on both w/h
    batch_size, n_channels, layer_height, layer_width = layer.size()
    xy1 = (layer_width - max_width) // 2
    xy2 = (layer_height - max_height) // 2
    return layer[:, :, xy2:(xy2 + max_height), xy1:(xy1 + max_width)]

class DenseLayer(nn.Sequential):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_features=in_channels))
        self.add_module('relu', nn.ReLU(inplace=True))
        
        #author's impl - lasange 'same' pads with half 
        # filter size (rounded down) on "both" sides
        self.add_module('conv', nn.Conv2d(in_channels=in_channels, 
                out_channels=growth_rate, kernel_size=3, stride=1, 
                  padding=1, bias=True))
        
        self.add_module('drop', nn.Dropout2d(0.2))

    def forward(self, x):
        return super(DenseLayer, self).forward(x)

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, n_layers, upsample=False):
        super(DenseBlock, self).__init__()
        self.upsample = upsample
        self.layers = nn.ModuleList([DenseLayer(
            in_channels + i*growth_rate, growth_rate)
            for i in range(n_layers)])
        
    def forward(self, x):
        if self.upsample:
            new_features = []
            #we pass all previous activations into each dense layer normally
            #But we only store each dense layer's output in the new_features array
            for layer in self.layers:
                out = layer(x)
                x = torch.cat([x, out], 1)
                new_features.append(out)
            return torch.cat(new_features,1)
        else:
            for layer in self.layers:
                out = layer(x)
                x = torch.cat([x, out], 1) # 1 = channel axis
            return x 
    
class TransitionDown(nn.Sequential):
    def __init__(self, in_channels):
        super(TransitionDown, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_features=in_channels))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(in_channels=in_channels, 
              out_channels=in_channels, kernel_size=1, stride=1, 
                padding=0, bias=True))
        self.add_module('drop', nn.Dropout2d(0.2))
        self.add_module('maxpool', nn.MaxPool2d(2))
        
    def forward(self, x):
        return super(TransitionDown, self).forward(x)
    
class TransitionUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionUp, self).__init__()
        self.convTrans = nn.ConvTranspose2d(in_channels=in_channels, 
               out_channels=out_channels, kernel_size=3, stride=2, 
              padding=0, bias=True) #crop = 'valid' means padding=0. Padding has reverse effect for transpose conv (reduces output size)
        #http://lasagne.readthedocs.io/en/latest/modules/layers/conv.html#lasagne.layers.TransposedConv2DLayer
        #self.updample2d = nn.UpsamplingBilinear2d(scale_factor=2)
        
    def forward(self, x, skip):
        out = self.convTrans(x)
        out = center_crop(out, skip.size(2), skip.size(3))
        out = torch.cat([out, skip], 1)
        return out
    
class Bottleneck(nn.Sequential):
    def __init__(self, in_channels, growth_rate, n_layers):
        super(Bottleneck, self).__init__()
        self.add_module('bottleneck', DenseBlock(in_channels, growth_rate, n_layers, upsample=True))

    def forward(self, x):
        return super(Bottleneck, self).forward(x)

## Model

In [4]:
class FCDenseNet(nn.Module):
    def __init__(self, in_channels=1, down_blocks=(5,5,5,5,5), 
                 up_blocks=(5,5,5,5,5), bottleneck_layers=5, 
                 growth_rate=16, out_chans_first_conv=48, n_classes=3):
        super(FCDenseNet, self).__init__()
        self.down_blocks = down_blocks
        self.up_blocks = up_blocks
        
        cur_channels_count = 0
        skip_connection_channel_counts = []
        
        
        #####################
        # First Convolution #
        #####################

        self.add_module('firstconv', nn.Conv2d(in_channels=in_channels, 
                  out_channels=out_chans_first_conv, kernel_size=3, 
                  stride=1, padding=1, bias=True))
        cur_channels_count = out_chans_first_conv
        
        
        
        #####################
        # Downsampling path #
        #####################
        
        self.denseBlocksDown = nn.ModuleList([])
        self.transDownBlocks = nn.ModuleList([])
        for i in range(len(down_blocks)):
            self.denseBlocksDown.append(
                DenseBlock(cur_channels_count, growth_rate, down_blocks[i]))
            cur_channels_count += (growth_rate*down_blocks[i])
            skip_connection_channel_counts.insert(0,cur_channels_count)
            self.transDownBlocks.append(TransitionDown(cur_channels_count))
            
            
            
        #####################
        #     Bottleneck    #
        #####################
        
        self.add_module('bottleneck',Bottleneck(cur_channels_count, 
                                     growth_rate, bottleneck_layers))
        prev_block_channels = growth_rate*bottleneck_layers
        cur_channels_count += prev_block_channels 
        
        
        
        #######################
        #   Upsampling path   #
        #######################

        self.transUpBlocks = nn.ModuleList([])
        self.denseBlocksUp = nn.ModuleList([])
        for i in range(len(up_blocks)-1):
            self.transUpBlocks.append(TransitionUp(prev_block_channels, prev_block_channels))
            cur_channels_count = prev_block_channels + skip_connection_channel_counts[i]

            self.denseBlocksUp.append(DenseBlock(
                cur_channels_count, growth_rate, up_blocks[i], 
                    upsample=True))
            prev_block_channels = growth_rate*up_blocks[i]
            cur_channels_count += prev_block_channels

            
        #One final dense block
        self.transUpBlocks.append(TransitionUp(
            prev_block_channels, prev_block_channels))
        cur_channels_count = prev_block_channels + skip_connection_channel_counts[-1]

        self.denseBlocksUp.append(DenseBlock(
            cur_channels_count, growth_rate, up_blocks[-1], 
                upsample=False))
        cur_channels_count += growth_rate*up_blocks[-1]

        
        
        #####################
        #      Softmax      #
        #####################

        self.finalConv = nn.Conv2d(in_channels=cur_channels_count, 
               out_channels=n_classes, kernel_size=1, stride=1, 
                   padding=0, bias=True)
        self.softmax = nn.LogSoftmax()
        
    def forward(self, x):
        #print("INPUT",x.size())
        out = self.firstconv(x)
        
        skip_connections = []
        for i in range(len(self.down_blocks)):
            #print("DBD size",out.size())
            out = self.denseBlocksDown[i](out)
            skip_connections.append(out)
            out = self.transDownBlocks[i](out)
            
        out = self.bottleneck(out)
        #print ("bnecksize",out.size())
        for i in range(len(self.up_blocks)):
            skip = skip_connections.pop()
            #print("DOWN_SKIP_PRE_UPSAMPLE",out.size(),skip.size())
            out = self.transUpBlocks[i](out, skip)
            #print("DOWN_SKIP_AFT_UPSAMPLE",out.size(),skip.size())
            out = self.denseBlocksUp[i](out)
            
        out = self.finalConv(out)
        out = self.softmax(out)
        return out
    
def FCDenseNet57(n_classes):
    return FCDenseNet(in_channels=1, down_blocks=(4, 4, 4, 4, 4), 
                 up_blocks=(4, 4, 4, 4, 4), bottleneck_layers=4, 
                 growth_rate=12, out_chans_first_conv=48, n_classes=n_classes)

def FCDenseNet67(n_classes):
    return FCDenseNet(in_channels=1, down_blocks=(5, 5, 5, 5, 5), 
                 up_blocks=(5, 5, 5, 5, 5), bottleneck_layers=5, 
                 growth_rate=16, out_chans_first_conv=48, n_classes=n_classes)

def FCDenseNet103(n_classes):
    return FCDenseNet(in_channels=1, down_blocks=(4,5,7,10,12), 
                 up_blocks=(12,10,7,5,4), bottleneck_layers=15, 
                 growth_rate=16, out_chans_first_conv=48, n_classes=n_classes)

## Predict

In [5]:
model_path = '/home/davince/Dropbox (OIST)/Deeplearning_system/OIST_fcdensenet_pytorch/best_model.pth'
data_directory = '/home/davince/Dropbox (OIST)/Deeplearning_system/OIST_fcdensenet_pytorch/DeepCell/RawImages/'



OUT_DIR = '/home/davince/Dropbox (OIST)/Deeplearning_system/OIST_fcdensenet_pytorch/20180516predict/'

#  Training Hyperparameters

N_CLASSES = 3


path = '/home/davince/Dropbox (OIST)/Deeplearning_system/OIST_fcdensenet_pytorch/DeepCell/RawImages/'

loader = DataLoader(path)

snapshot = torch.load(model_path)

model = FCDenseNet57(N_CLASSES)
model.load_state_dict(snapshot['state_dict'])
model.cuda()

rows 4
cols 4
offset w 37
offset h 37


UnicodeDecodeError: 'ascii' codec can't decode byte 0x80 in position 0: ordinal not in range(128)

In [7]:
def get_predictions(outputs):
    return np.argmax(outputs, 0)


def save_predictions(out_dir, name, prediction, boundary_dir, cytoplasm_dir):   
        
    boundary = np.zeros(prediction.shape, dtype=np.uint8)
    cytoplasm = np.zeros(prediction.shape, dtype=np.uint8)
        
    boundary[np.where(prediction == 1)] = 255
    cytoplasm[np.where(prediction == 2)] = 255
        
    cv2.imwrite(op.join(boundary_dir, name+'boundary.png'), boundary)
    cv2.imwrite(op.join(cytoplasm_dir, name+'cytoplasm.png'), cytoplasm)
        
        
def get_predictions_variable(outputs):
    bs, c, h, w = outputs.size()
    tensor = outputs.data
    values, indices = tensor.cpu().max(1)
    indices = indices.view(bs, h, w)
    return indices
        

def predict(model, loader, out_dir):
    torch.cuda.empty_cache()
    model.eval()
    boundary_dir = op.join(out_dir, "boundary")
    if not op.exists(boundary_dir):
        os.makedirs(boundary_dir)
    cytoplasm_dir = op.join(out_dir, "cytoplasm")
    if not op.exists(cytoplasm_dir):
        os.makedirs(cytoplasm_dir)
    
    
    with tqdm(total=loader.batch_num) as pbar:
        for i, data in enumerate(loader):
            name, inputs = data
            sample_prediction = []
            for input in inputs:
                outputs = model(Variable(input).cuda())
    
                outputs = outputs.data.cpu()
                sample_prediction.append(outputs)
                torch.cuda.empty_cache()
            full_predictions = loader.recreate_mask(sample_prediction)
                
            save_predictions(out_dir, name, get_predictions(full_predictions), boundary_dir, cytoplasm_dir)
            
            pbar.update(1)
            torch.cuda.empty_cache()

In [8]:
%%time
predict(model, loader, OUT_DIR)

100%|██████████| 148/148 [03:47<00:00,  1.49s/it]

CPU times: user 1min 22s, sys: 2min 7s, total: 3min 30s
Wall time: 3min 47s
